In [ ]:
"""
Simple Paystack initialize + verify example (requests-based).
Replace PAYSTACK_SECRET_KEY with your test or live secret key.
"""

import uuid
import time
import requests

PAYSTACK_SECRET_KEY = "sk_test_827af86e7e94b2652558f3ff77f9ef3883f0fe5e"  # <-- replace with your secret key
PAYSTACK_INITIALIZE_URL = "https://api.paystack.co/transaction/initialize"
PAYSTACK_VERIFY_URL = "https://api.paystack.co/transaction/verify/{}"

HEADERS = {
    "Authorization": f"Bearer {PAYSTACK_SECRET_KEY}",
    "Content-Type": "application/json",
}


def generate_reference(prefix: str = "ORDER") -> str:
    """
    Generate a reasonably unique transaction reference.
    Format: <prefix>-<uuid4>-<timestamp>
    """
    return f"{prefix}-{uuid.uuid4().hex[:8]}-{int(time.time())}"


def initialize_transaction(amount_ngn: float, email: str, callback_url: str = None, reference: str = None):
    """
    Initialize a Paystack transaction.
    amount_ngn: amount in Naira (float or int). We'll convert to kobo for Paystack.
    email: customer's email (required by Paystack).
    callback_url: optional. Paystack will redirect here after payment.
    reference: optional custom reference (if None, we'll generate one).
    Returns JSON response (dict) from Paystack.
    """
    if reference is None:
        reference = generate_reference()

    amount_kobo = int(round(amount_ngn * 100))  # convert to kobo (integer)
    payload = {
        "email": email,
        "amount": amount_kobo,
        "reference": reference,
    }
    if callback_url:
        payload["callback_url"] = callback_url

    resp = requests.post(PAYSTACK_INITIALIZE_URL, json=payload, headers=HEADERS)
    resp.raise_for_status()
    data = resp.json()

    # data structure: { "status": true, "message":"...", "data": { authorization_url, access_code, reference } }
    return data


def verify_transaction(reference: str):
    """
    Verify a transaction by reference.
    Returns JSON response (dict) from Paystack.
    """
    url = PAYSTACK_VERIFY_URL.format(reference)
    resp = requests.get(url, headers=HEADERS)
    resp.raise_for_status()
    return resp.json()


# -------------------------
# Example usage:
# -------------------------
if __name__ == "__main__":
    TEST_EMAIL = "customer@example.com"
    AMOUNT_NGN = 1500.00  # Naira
    CALLBACK = "https://yourdomain.com/paystack-callback"  # optional

    # 1) Initialize
    init_resp = initialize_transaction(AMOUNT_NGN, TEST_EMAIL, callback_url=CALLBACK)
    if init_resp.get("status"):
        auth_url = init_resp["data"]["authorization_url"]
        reference = init_resp["data"]["reference"]
        print("Open this URL to complete payment:", auth_url)
        print("Reference:", reference)
    else:
        print("Failed to initialize:", init_resp)

    # 2) Later, after payment attempt, verify by reference
    # verify_resp = verify_transaction(reference)
    # print("Verify response:", verify_resp)


Open this URL to complete payment: https://checkout.paystack.com/187ovhvci2iukgp
Reference: ORDER-f8438312-1760952279
